In [1]:
import json
import os
from os import listdir
import pandas
from tqdm import tqdm as tqdm
import re

In [2]:
import spacy
nlp = spacy.load('en_core_web_md')
#doc = nlp("Hello how are you !")

In [3]:
path = "Ann"
files = [ f for f in listdir(path) if f.split('.')[1] == 'csv']
print (files)

['6.csv', '7.csv', '5.csv', '4.csv', '1.csv', '3.csv', '2.csv', '14.csv', '10.csv', '11.csv', '13.csv', '12.csv', '9.csv', '8.csv']


In [4]:
allData = []
for eachFile in files:
    base = os.path.basename(eachFile).split('.')[0]
    #print (base)
    ff = os.path.join(path,eachFile)
    data = pandas.read_csv(ff,header = None,encoding="utf-8")
#     print (data)
#     input()
    data[[3]] = data[[3]].fillna('-')
    data[[4]] = data[[4]].fillna('-')
    data = data.values.tolist()
    allData.append(data)

In [5]:
doc = nlp("It can be seen")
for g in doc:
    print (g,g.tag_)
print (len(allData))

It PRP
can MD
be VB
seen VBN
14


In [6]:
# def nodeAccuracy(output):
#     ''' Calculates the accuracy of the graph nodes '''
    
#     total = 0
#     correct = 0
#     for each in output:
#         gold,pred = each[0],each[1]
#         #print (gold,pred,each)
#         if gold != '-':
#             total += 1
#             if pred != '-/-':
#                 if (pred == gold):
#                     correct += 1
#                 else:
#                     pred = pred.split('/')
#                     for i in pred:
#                         if i in gold and len(gold)==1:
#                             correct += 1
#     accuracy = round(correct / total,2)
#     print (total,correct,accuracy)


In [26]:
def nodeAccuracy(output):
    ''' Calculates the accuracy of the graph nodes '''
    
    # Penalize half if partially correct
    total = 0.0
    correct = 0.0
    for each in output:
        total += 1.0
        gold,pred = each[0],each[1]
        if gold == pred:
            correct+=1.0
        elif ((gold in pred) or (pred in gold)) and gold and pred:
            correct+=0.5
        else:
            correct+=0.0
#         print (gold,pred,correct)
    accuracy = round(correct / total,2)
    print ("Total Number of Actions/Informations:",total,"\nCorrect Prediction:",correct,"\nAccuracy:",accuracy,"%")
            

In [27]:
# actionRules = ['PRP-MD-VB','PRP-RB-VB','PRP-VB',
#                 'PRP-MD-VBG','PRP-RB-VBG','PRP-VBG',
#                 'PRP-MD-VBN','PRP-RB-VBN','PRP-VBN',
#                 'PRP-MD-VBP','PRP-RB-VBP','PRP-VBP',
#                'PRP-MD-RB-VB'
#                 #'PRP-MD-VBZ','PRP-RB-VBZ','PRP-VBZ',
#                 'PRP-MD-VBD','PRP-RB-VBD','PRP-VBD']
actionRules = ["PRP-MD-VB","PRP-RB-VB","PRP-VB","PRP-MD-RB-VB"]
# infoRules = ['EX-VBD','EX-VBP','EX-VBZ','VBD-IN','PRP-VBZ',
#             'NN-VBD-VBN','NNS-VBD-VBN','NN-VBD-VBG','NNS-VBD-VBG',
#             'NN-VBP-VBN','NNS-VBP-VBN','NN-VBP-VBG','NNS-VBP-VBG',
#             'NN-VBZ-VBN','NNS-VBZ-VBN',
#             'NN-MD-VB','NN-MD-RB-VB']

infoRules = ["NN-VB-NN","NN-VB-VB","NN-VB-RB-NN","NN-RB-VB-NN","NN-MD-VB","NN-MD-RB-VB","EX-VB"]
filterPos = 'PDT|DT|IN|,|:|\.|JJ|SYM|TO|HYPH'
output = []
#print (allData[1])
for eachFile in allData:
    for docid,lineno,content,enttype,edges in tqdm(eachFile):
#         print (docid,lineno,content,enttype,edges)
        #input()
        if enttype.strip()=="-":
            continue
        posTag = []
        for word in nlp(content):
            posTag.append(word.tag_)
        posTagSeq = "-".join(posTag)
        # Removing Adverbs(RB%, RP%), Adjective(PDT,JJ%), Conjunction &support preposition(IN), Determiner(DT), Punctuation (,:.)
        
        reducedPosTagSeq = [x for x in posTag if not re.search(filterPos,x) ]
        reducedPosTagSeqMod = []
        for x in reducedPosTagSeq:
            if re.match("NN*",x):
                reducedPosTagSeqMod.append("NN")
            elif re.match("VB*",x):
                reducedPosTagSeqMod.append("VB")
            else:
                reducedPosTagSeqMod.append(x)
                
#         print (reducedPosTagSeq)
#         print (reducedPosTagSeqMod)
        posTagSeqNew = "-".join(reducedPosTagSeqMod)

#         print ("reducedPosTagSeq:",reducedPosTagSeq)
#         print (posTag,posTagSeq)
        predEnttypeAction = ''
        for eachRule in actionRules:
            if eachRule in posTagSeqNew:
                predEnttypeAction = 'A'
#                 print ("Rule:",eachRule)
                break
                
        predEnttypeInfo = ''
        for eachRule in infoRules:
            if eachRule in posTagSeqNew:
                predEnttypeInfo = 'I'
#                 print ("Rule:",eachRule)
                break

        if enttype in ["A,I","I,A","M"]:
            enttype = "A/I"
        pred=""
        if predEnttypeAction and predEnttypeInfo:
            pred = predEnttypeAction+"/"+predEnttypeInfo
        else:
            pred = predEnttypeAction+predEnttypeInfo
            
        output.append([enttype,pred])
#         print ("Gold:",enttype, "Pred:",predEnttypeAction,predEnttypeInfo,",",pred)
        #input()
nodeAccuracy(output)

100%|██████████| 36/36 [00:00<00:00, 165.67it/s]

Total Number of Actions/Informations: 517.0 
Correct Prediction: 284.5 
Accuracy: 0.55 %
